In [1]:
from divergence_plotting import *
from basic_plotting import get_target_results
from universal_divergence import estimate

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

WDIR = Path().cwd().parent

Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).
Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


In [2]:
_dirs = [
    "WASP-19b",
    "WASP-17b",
    "WASP-12b",
    "HD-189733b",
    "HAT-P-26b",
    "HAT-P-12b",
    "HAT-P-1b",
    # "WASP-39b",
    # "WASP-121b",
]

# dirs = [WDIR / "data/retrievals" / d for d in _dirs]

dirs = [WDIR / "data/synthetic_spectra" / d for d in _dirs]

planets = []

for d in dirs:
    # par_names, traces, weights, truths, subsamples, divergences_truth, cross_divergences
    divs = get_high_weight_subsample(d)

    planets.append(divs)



In [5]:
data = np.array(planets, dtype=object).T
print(data.shape)

(7, 7)


In [7]:
def plot_divergence_par(par_name, data):

    # fig, ax = plt.subplots(1,1)

    for p in range(data.shape[1]):

        print(data[0, p])

        if not par_name in data[0, p]:
            continue

        idx = np.argwhere(data[0, p] == par_name)

        truth = data[3, p][idx]
        divergence_truth = data[5, p][par_name]
        cross_divergences = data[6, p][par_name]

        print(truth, divergence_truth, cross_divergences)

        # ax.scatter()

plot_divergence_par("ace_co_ratio", data)

[array(['planet_radius', 'T', 'ace_metallicity', 'ace_co_ratio'],
      dtype='<U15'), array(['planet_radius', 'T', 'ace_metallicity', 'ace_co_ratio',
       'Offset_2'], dtype='<U15'), array(['planet_radius', 'T', 'ace_metallicity', 'ace_co_ratio'],
      dtype='<U15')]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()